# Librerias

In [2]:
import pandas as pd
import numpy as np
import random
import math

## Ingresamos la matriz de datos

In [3]:
alternatives = np.array([[7,15,12,34,9,2,5,8,40,17]
,[5,14,20,26,9,2,4,15,22,16],
[11,19,21,31,1,11,4,42,14,18],
[13,1,24,27,35,6,1,5,4,2]])
#Convertimos los datos de entrada en una matriz
pd_alter = pd.DataFrame(data = alternatives,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns=['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'])
pd_alter

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10
A1,7,15,12,34,9,2,5,8,40,17
A2,5,14,20,26,9,2,4,15,22,16
A3,11,19,21,31,1,11,4,42,14,18
A4,13,1,24,27,35,6,1,5,4,2


# <center> Punto 1A

### Implementación ELECTRE
#### <span style="color:Blue"> Paso 1:

$En\ primer\ lugar\ le\ damos\ los\ pesos\ a\ cada\ criterio\ para\ construir\ la\ matriz\ de\ desión$

Para esto se decidio hacerlo de manera aleatoria ya que no se tiene preferencia sobre ninguna

In [4]:
n = 10
nums = [random.random() for _ in range(n)]
suma = sum(nums)
weigth = [num/suma for num in nums]
w = [0.16080725415783573,
0.20565938230877176,
0.1785406855161036,
0.0321902504032635,
0.0289215951779111,
0.137201741307777,
0.1315644222831187,
0.03016823558028442,
0.06923148936889213,
0.02571494389604221]

#### <span style="color:Blue"> Paso 3:

$Construir\ la\ matriz\ de\ decisión\ $
$donde:$

$$v_i=\frac{r_i-min (r_i)}{max (r_i)-min (r_i)}$$


## Norma Electre


In [5]:
# Normalización de la matriz utilizando el método Electre
def normelectre(mat):
    sizes  = np.shape(mat)
    m = sizes[0]
    n = sizes[1]
    norm_matrix = np.zeros((m, n))
    for j in range(n):
        max_value = np.max(mat[:, j])
        min_value = np.min(mat[:, j])
        for i in range(m):
            if max_value == min_value:
                norm_matrix[i, j] = 1
            else:
                norm_matrix[i, j] = (mat[i, j]-min_value ) / (max_value - min_value)
    return norm_matrix
matriz = normelectre(alternatives)
pd_norm_matrix = pd.DataFrame(data=matriz,
                            index=['A1', 'A2', 'A3', 'A4'],
                            columns=['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'])

pd_norm_matrix


,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10
A1,0.25,0.777778,0.000000,1.000,0.235294,0.000000,1.00,0.081081,1.000000,0.9375
A2,0.00,0.722222,0.666667,0.000,0.235294,0.000000,0.75,0.270270,0.500000,0.8750
A3,0.75,1.000000,0.750000,0.625,0.000000,1.000000,0.75,1.000000,0.277778,1.0000
A4,1.00,0.000000,1.000000,0.125,1.000000,0.444444,0.00,0.000000,0.000000,0.0000


In [6]:
arr = pd_alter.agg(['min', 'max'])
ar =np.matrix(arr)
ar[1]-ar[0]


matrix([[ 8, 18, 12,  8, 34,  9,  4, 37, 36, 16]])

## Matriz de concordancia

$Esta\ matriz\ se\ define\ como:$
$$C(A,B)=\sum_{j|r_j(a)>r_j(b)} w_j+0.5\sum_{j|r_j(a)=r_j(b)} w_j$$


Y el indice esta dado por:

$$C_{ik}=\sum_{j:C_j(A_i)>C_j(A_k)} w_j+0.5\sum_{j|C_j(A_i)=C_j(A_k)} w_j$$



In [7]:
# Definir la función de concordancia utilizando el criterio de Pareto

# Construir la matriz de concordancia
def matriz_concordance(norm_matrix,w):
    sizes  = np.shape(norm_matrix)
    m = sizes[0]
    n = sizes[1]
    concordance_matrix = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            if i == j:
                concordance_matrix[i,j] = 0
            else:
                row_i = norm_matrix[i,:]
                row_j = norm_matrix[j,:]
                concordant_criteria = []
                for k in range(n):
                    if row_i[k]==row_j[k]:
                        concordant_criteria.append(w[k]/2)
                        val = sum(concordant_criteria)
                    if row_i[k]>row_j[k]:
                        concordant_criteria.append(w[k])
                        val = sum(concordant_criteria)
                concordance_matrix[i,j] = val
    return concordance_matrix
con = matriz_concordance(matriz,w)
pd_norm_matrix = pd.DataFrame(data=con,
                            index=['A1', 'A2', 'A3', 'A4'],
                            columns=['A1', 'A2', 'A3', 'A4'])

pd_norm_matrix


,A1,A2,A3,A4
A1,0.000000,0.708229,0.261908,0.494529
A2,0.291771,0.000000,0.163935,0.462338
A3,0.738092,0.836065,0.000000,0.631730
A4,0.505471,0.537662,0.368270,0.000000


$Construir\ la\ matriz\ de\ decisión\ ponderada $
$donde:$

$$\overline{v}_{ij} =v_{ij}\cdot w_j, \forall j = 1,2,...,n$$

$Donde:$
$$\sum W_j =1$$

In [8]:
def normalizepesos(vec, mat):
    sizes  = np.shape(mat)
    m = sizes[0]
    n = sizes[1]
    v = np.zeros((m, n))
    for j in range(n):
        for i in range(m):
                v[i][j] = mat[i][j] * vec[j]
    return v
normalizada=matriz
normalizadap = normalizepesos(w,normalizada)
pd_normp = pd.DataFrame(data = normalizadap,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns=['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'])
pd_normp

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10
A1,0.040202,0.159957,0.000000,0.032190,0.006805,0.000000,0.131564,0.002446,0.069231,0.024108
A2,0.000000,0.148532,0.119027,0.000000,0.006805,0.000000,0.098673,0.008154,0.034616,0.022501
A3,0.120605,0.205659,0.133906,0.020119,0.000000,0.137202,0.098673,0.030168,0.019231,0.025715
A4,0.160807,0.000000,0.178541,0.004024,0.028922,0.060979,0.000000,0.000000,0.000000,0.000000


## Matriz de discordancia
#### Esta matriz esta definida de la siguiente manera

$$D(i,k)= \frac{max_{(i,k)|\overline{v_j}(i)<\overline{v_j}(k)}\overline{v_j}(k)-\overline{v_j}(i)}{max_{\forall(i,k)}|\overline{v_j}(k)-\overline{v_j}(i)|}$$

In [9]:
from pandas._config.config import config_prefix
# Construir la matriz de disconcordancia
def matriz_disconcordance(norm_matrix,w):
    sizes  = np.shape(norm_matrix)
    m = sizes[0]
    n = sizes[1]
    disconcordance_matrix = np.zeros((m, m))
    for i in range(m):
        for k in range(m):
            if i == k:
                disconcordance_matrix[i,k] = 0
            else:
                row_i = norm_matrix[i,:]
                row_k = norm_matrix[k,:]
                disconcordant_criteria = []
                for j in range(n):
                  maxi = max(abs(row_i-row_k))
                  if row_i[j]<row_k[j]:
                    dif = max(row_i-row_k)
                    disconcordance_matrix[i,k] = dif/maxi
    return disconcordance_matrix
dis = matriz_disconcordance(normalizadap,w)
pd_norm_matrixp = pd.DataFrame(data=dis,
                            index=['A1', 'A2', 'A3', 'A4'],
                            columns=['A1', 'A2', 'A3', 'A4'])

pd_norm_matrixp


,A1,A2,A3,A4
A1,0.0,0.337753,0.364431,0.895915
A2,1.0,0.000000,0.112133,0.923663
A3,1.0,1.000000,0.000000,1.000000
A4,1.0,1.000000,0.217034,0.000000


In [33]:
#Información para el documento
row_i = normalizadap[1,:]
row_k = normalizadap[0,:]
row_k


array([0.04020181, 0.1599573 , 0.        , 0.03219025, 0.00680508,
       0.        , 0.13156442, 0.00244607, 0.06923149, 0.02410776])

# Definión de umbrales

$$aSb ⟺ C(a,b) \geq c^*\ y\ D(a,b) \leq d^*$$



In [11]:
elementos = 0
sumatoria = 0
for fila in dis:
    for elemento in fila:
        sumatoria += elemento
        elementos += 1

D = sumatoria / elementos
D

0.5531830942422321

In [12]:
elementos = 0
sumatoria = 0
for fila in con:
    for elemento in fila:
        sumatoria += elemento
        elementos += 1

c = sumatoria / elementos
c

0.37500000000000006

In [13]:
def domincon(c, mat):
    sizes  = np.shape(mat)
    m = sizes[0]
    n = sizes[1]
    v = np.zeros((m, n))
    for j in range(n):
      for i in range(m):
        if mat[i][j] <=c:
          v[i][j] = 0 
        else:
          v[i][j] = 1
    return v   
m_condordanciaDom = domincon(c,con)
condordanciaDom = pd.DataFrame(data = m_condordanciaDom,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns= ['A1','A2', 'A3', 'A4'])
condordanciaDom

,A1,A2,A3,A4
A1,0.0,1.0,0.0,1.0
A2,0.0,0.0,0.0,1.0
A3,1.0,1.0,0.0,1.0
A4,1.0,1.0,0.0,0.0


In [14]:
def domindis(d, mat):
    sizes  = np.shape(mat)
    m = sizes[0]
    n = sizes[1]
    v = np.zeros((m, n))
    for j in range(n):
      for i in range(m):
        if mat[i][j] >=d:
            v[i][j] = 1 
        else:
          v[i][j] = 0
    return v   
m_discondordanciaDom = domindis(D,dis)
discondordanciaDom = pd.DataFrame(data = m_discondordanciaDom,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns= ['A1','A2', 'A3', 'A4'])
discondordanciaDom

,A1,A2,A3,A4
A1,0.0,0.0,0.0,1.0
A2,1.0,0.0,0.0,1.0
A3,1.0,1.0,0.0,1.0
A4,1.0,1.0,0.0,0.0


In [15]:
def domin_agg(dis, con):
    sizes  = np.shape(dis)
    m = sizes[0]
    n = sizes[1]
    v = np.zeros((m, n))
    for j in range(n):
      for i in range(m):
        v[i][j] = dis[i][j]*con[i][j]
    return v   
domin_agg = domin_agg(m_discondordanciaDom,m_condordanciaDom)
domin_agg_m = pd.DataFrame(data = domin_agg,
                        index = ['A1','A2', 'A3', 'A4'],
                        columns= ['A1','A2', 'A3', 'A4'])
domin_agg_m

,A1,A2,A3,A4
A1,0.0,0.0,0.0,1.0
A2,0.0,0.0,0.0,1.0
A3,1.0,1.0,0.0,1.0
A4,1.0,1.0,0.0,0.0
